# PYTHON PROGRAMMING QUESTION

Python is the preferred programming language that we are using for multiple purpose (API connections, Modelling, Data engineering)

1. Write a twitter API call to download tweets for 3 - 4 stock market ticker symbol as a hashtag in python
2. Use downloaded data and perform data cleaning using Python (library functions or regex)
3. Basic visualization of the data you cleaned (No of tweets for particular ticker symbol in day/hour, No of users tweeting in a day/hour for ticker symbol)


In [216]:
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
import tweepy
import configparser

### Major Canadian Financial Institutions Ticker Lookup

In [214]:
tickers = ['$BMO', '$TD', '$RBC']

In [208]:
# read configs
config = configparser.ConfigParser()
config.read('config.ini')

api_key = config['twitter']['api_key']
api_key_secret = config['twitter']['api_key_secret']

access_token = config['twitter']['access_token']
access_token_secret = config['twitter']['access_token_secret']

# authentication
auth = tweepy.OAuthHandler(api_key, api_key_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

# user tweets
# user = 'veritasium'
# limit=300

# tweets = tweepy.Cursor(api.user_timeline, screen_name=user, count=200, tweet_mode='extended').items(limit)

# search tweets
tickers = ['$BMO', '$TD', '$RY']
limit=300
df = []
for ticker in tickers:
    tweets = tweepy.Cursor(api.search_tweets, q=ticker, count=100, tweet_mode='extended').items(limit)

    # tweets = api.user_timeline(screen_name=user, count=limit, tweet_mode='extended')

    # create DataFrame
    columns = ['Created At', 'User', 'Tweet', 'Location']
    data = []

    for tweet in tweets:
        data.append([tweet.created_at, tweet.user.screen_name, tweet.full_text, tweet.user.location])

    intermediate_df = pd.DataFrame(data, columns=columns)
    query_labels = pd.Series([ticker]).repeat(len(intermediate_df))

    query_labels_df = query_labels.to_frame(name="Tickers").reset_index(drop=True)

    df = pd.concat([intermediate_df, query_labels_df], axis=1)

#   print(df)



In [209]:
df[df['Tickers']=='$RY']

,Created At,User,Tweet,Location,Tickers
0,2022-05-29 23:44:03+00:00,Focus_on_Dvds,Dividend Increases Week 21 $ASH $CM $CTO $DCI ...,Global,$RY
1,2022-05-29 23:05:03+00:00,AlertTrade,🔴🔴 You are invited to test drive TradeIdeas ...,,$RY
2,2022-05-29 21:09:56+00:00,silberschmelzer,@wildace @SilbergleitJr One lesson we learned ...,,$RY
3,2022-05-29 18:54:44+00:00,rosu_milescu,"RT @rosu_milescu: ALL $RY""$CUM🚻\""\nunder CYA:N...","LanglEY,PARk VA",$RY
4,2022-05-29 18:54:10+00:00,rosu_milescu,"ALL $RY""$CUM🚻\""\nunder CYA:NYC🚻\nStatION,DC:La...","LanglEY,PARk VA",$RY
...,...,...,...,...,...
295,2022-05-26 10:31:42+00:00,QuantchaIdeas,StockTwits Trending Alert: Trading recent inte...,,$RY
296,2022-05-26 10:18:14+00:00,Tickeron,$RY reports Earnings today. $RY in Uptrend: RS...,"Sunnyvale, CA",$RY
297,2022-05-26 10:15:14+00:00,mediasentiment,$RY new alert at https://t.co/A7qrDarJHY #st...,,$RY
298,2022-05-26 10:14:44+00:00,mediasentiment,$RY new alert at https://t.co/A7qrDarJHY #st...,,$RY


# Basic data cleaning

In [215]:
# Create a function to clean the tweets
def cleanTxt(text):
    text = re.sub('@[A-Za-z0–9]+', '', text) #Removing @mentions
    text = re.sub('#', '', text) # Removing '#' hash tag
    text = re.sub('RT[\s]+', '', text) # Removing RT
    text = re.sub('https?:\/\/\S+', '', text) # Removing hyperlink
    return text
df["Tweet"] = df["Tweet"].apply(cleanTxt)
df

,Created At,User,Tweet,Location,Tickers
0,2022-05-29 23:44:03+00:00,Focus_on_Dvds,Dividend Increases Week 21 $ASH $CM $CTO $DCI ...,Global,$RY
1,2022-05-29 23:05:03+00:00,AlertTrade,🔴🔴 You are invited to test drive TradeIdeas ...,,$RY
2,2022-05-29 21:09:56+00:00,silberschmelzer,One lesson we learned from the 2008 global f...,,$RY
3,2022-05-29 18:54:44+00:00,rosu_milescu,"_milescu: ALL $RY""$CUM🚻\""\nunder CYA:NYC🚻\nSta...","LanglEY,PARk VA",$RY
4,2022-05-29 18:54:10+00:00,rosu_milescu,"ALL $RY""$CUM🚻\""\nunder CYA:NYC🚻\nStatION,DC:La...","LanglEY,PARk VA",$RY
...,...,...,...,...,...
295,2022-05-26 10:31:42+00:00,QuantchaIdeas,StockTwits Trending Alert: Trading recent inte...,,$RY
296,2022-05-26 10:18:14+00:00,Tickeron,$RY reports Earnings today. $RY in Uptrend: RS...,"Sunnyvale, CA",$RY
297,2022-05-26 10:15:14+00:00,mediasentiment,$RY new alert at stocks daytrading NYSE NA...,,$RY
298,2022-05-26 10:14:44+00:00,mediasentiment,$RY new alert at stocks daytrading NYSE NA...,,$RY


In [212]:
#Sentiment Analysis
def percentage(part,whole):
    return 100 * float(part)/float(whole)

#Assigning Initial Values
positive = 0
negative = 0
neutral = 0
#Creating empty lists
tweet_list1 = []
neutral_list = []
negative_list = []
positive_list = []

#Iterating over the tweets in the dataframe
for tweet in df['Tweet']:
    tweet_list1.append(tweet)
    analyzer = SentimentIntensityAnalyzer().polarity_scores(tweet)
    neg = analyzer['neg']
    neu = analyzer['neu']
    pos = analyzer['pos']
    comp = analyzer['compound']

    if neg > pos:
        negative_list.append(tweet) #appending the tweet that satisfies this condition
        negative += 1 #increasing the count by 1
    elif pos > neg:
        positive_list.append(tweet) #appending the tweet that satisfies this condition
        positive += 1 #increasing the count by 1
    elif pos == neg:
        neutral_list.append(tweet) #appending the tweet that satisfies this condition
        neutral += 1 #increasing the count by 1 

positive = percentage(positive, len(df)) #percentage is the function defined above
negative = percentage(negative, len(df))
neutral = percentage(neutral, len(df))


In [213]:
#Converting lists to pandas dataframe
tweet_list1 = pd.DataFrame(tweet_list1)
neutral_list = pd.DataFrame(neutral_list)
negative_list = pd.DataFrame(negative_list)
positive_list = pd.DataFrame(positive_list)
#using len(length) function for counting
print("Since " + noOfDays + " days, there have been", len(tweet_list1) ,  "tweets on " + query, end='\n*')
print("Positive Sentiment:", '%.2f' % len(positive_list), end='\n*')
print("Neutral Sentiment:", '%.2f' % len(neutral_list), end='\n*')
print("Negative Sentiment:", '%.2f' % len(negative_list), end='\n*')


Since 30 days, there have been 300 tweets on $BMO
*Positive Sentiment: 59.00
*Neutral Sentiment: 231.00
*Negative Sentiment: 10.00
*

## Ideas

In [150]:
import tweepy
import configparser
import pandas as pd

# read configs
config = configparser.ConfigParser()
config.read('config.ini')

api_key = config['twitter']['api_key']
api_key_secret = config['twitter']['api_key_secret']

access_token = config['twitter']['access_token']
access_token_secret = config['twitter']['access_token_secret']

# authentication
auth = tweepy.OAuthHandler(api_key, api_key_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

# user tweets
# user = 'veritasium'
# limit=300

# tweets = tweepy.Cursor(api.user_timeline, screen_name=user, count=200, tweet_mode='extended').items(limit)

# search tweets
keywords = '$RY'
limit=300

tweets = tweepy.Cursor(api.search_tweets, q=keywords, count=100, tweet_mode='extended').items(limit)

# tweets = api.user_timeline(screen_name=user, count=limit, tweet_mode='extended')

# create DataFrame
columns = ['User', 'Tweet']
data = []

for tweet in tweets:
    data.append([tweet.user.screen_name, tweet.full_text])

df = pd.DataFrame(data, columns=columns)

print(df)



                User                                              Tweet
0    silberschmelzer  @wildace @SilbergleitJr One lesson we learned ...
1       rosu_milescu  RT @rosu_milescu: ALL $RY"$CUM🚻\"\nunder CYA:N...
2       rosu_milescu  ALL $RY"$CUM🚻\"\nunder CYA:NYC🚻\nStatION,DC:La...
3       rosu_milescu  RT @rosu_milescu: ALL $RY"$CUM🚻\"\nunder CYA:N...
4       rosu_milescu  ALL $RY"$CUM🚻\"\nunder CYA:NYC🚻\nStatION,DC:La...
..               ...                                                ...
295   mediasentiment  $RY  new alert at https://t.co/A7qrDarJHY  #st...
296   mediasentiment  $RY  new alert at https://t.co/A7qrDarJHY  #st...
297    ChettaCheetah  RT @MarketCurrents: $RY - Royal Bank of Canada...
298  ShortAlgoTrades  $RY Trading Ideas | Awaiting Short signal. 85....
299   MarketCurrents  $RY - Royal Bank of Canada raises dividend by ...

[300 rows x 2 columns]


# SQL ANALYTICS QUESTION

1. Use your own GCP account
2. Get sample data from BQ

- GA Dataset - https://support.google.com/analytics/answer/4419694?hl=en#zippy=%2Cin-this-article

3. BQ analytical queries

- BQ Cookbook - https://support.google.com/analytics/answer/7586738?hl=en#zippy=%2Cin-this-article

4. **Question**: How do you derive customers insights from GA data (feel free to perform data integration and build reprots!)

5. **Evaluation criteria for this exercise**: 
- SQL, 
- Working with nested fields, 
- Analytical expertise

6. **Deliverable artifacts**: Powerpoint deck summarizing methodology and findings along with queries

---------------------


### Answer:


### Problem statement
The dataset provides 12 months (August 2016 to August 2017) of obfuscated Google Analytics 360 data from the Google Merchandise Store , a real ecommerce store that sells Google-branded merchandise, in BigQuery. It’s a great way analyze business data and learn the benefits of using BigQuery to analyze Analytics 360 data Learn more about the data 
The data includes The data is typical of what an ecommerce website would see and includes the following information:

- Traffic source data: information about where website visitors originate, including data about organic traffic, paid search traffic, and display traffic
- Content data: information about the behavior of users on the site, such as URLs of pages that visitors look at, how they interact with content, etc.
- Transactional data: information about the transactions on the Google Merchandise Store website.

Data for some fields is obfuscated such as fullVisitorId, or removed such as clientId, adWordsClickInfo and geoNetwork. “Not available in demo dataset” will be returned for STRING values and “null” will be returned for INTEGER values when querying the fields containing no data.

**Additional details**

Type: Datasets
Category: Advertising, Analytics

Dataset source: Google Merchandise Store

Cloud service: BigQuery

Expected update frequency: Historical

**About GA Sessions**: For each Analytics view that is enabled for BigQuery integration, a dataset is added using the view ID as the name.

**Schema description**: https://support.google.com/analytics/answer/3437719?hl=en&ref_topic=3416089

In [223]:
!export GOOGLE_APPLICATION_CREDENTIALS="uol-datascience-341918-32809f524c18.json"

In [222]:
# Authentication
import os
from google.cloud import bigquery
from google.oauth2 import service_account
# Set environment variable
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="./uol-datascience-341918-32809f524c18.json"
# Project name
project_id = "uol-datascience"
# Path to the key
key_path="./uol-datascience-341918-32809f524c18.json"
#Credentials
credentials = service_account.Credentials.from_service_account_file(
    key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"],
)
#Instantiate BQ client
client = bigquery.Client(credentials=credentials, project=credentials.project_id)
# ROCK AND ROLLLLLL

In [224]:
# Perform a query.
q = (
"""
SELECT  fullVisitorId,
        visitNumber,
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_20170710` LIMIT 1000
"""
)
query_job = client.query(q)  # API request
rows = query_job.result()  # Waits for query to finish, ADD CHECKS FOR FAILED QUERY

#  Create a pandas DataFrame by loading all pages of a query 
# https://googleapis.dev/python/bigquery/latest/generated/google.cloud.bigquery.table.RowIterator.html
rows_df = rows.to_dataframe(create_bqstorage_client=True) 

In [225]:
print(rows_df)

           fullVisitorId  visitNumber
0    6131429996146284901            1
1    6332799276550005754            1
2    1217531208632585477            1
3     358414482172594579            1
4    1631409567299872036            1
..                   ...          ...
995   359543390044601586            1
996  3590334566572045711            1
997  9515230126025032270            1
998  4492737792188874011            1
999  6708420619872985019            1

[1000 rows x 2 columns]


### 1. Show the session count for all visitors who landed on our website on July 10, 2017. 

In [ ]:
# The unique visitor ID
# The session number for this user. If this is the first session, then this is set to 1.
"""
SELECT  fullVisitorId,
        visitNumber,
 FROM `bigquery-public-data.google_analytics_sample.ga_sessions_20170710` LIMIT 1000
"""

### 2. Summary of daily session statistics (concat df, axis=0) in the last x days/months/years 

totals: This section contains aggregate values across the session

totals.visits: The number of sessions (for convenience). This value is 1 for sessions with interaction events. The value is null if there are no interaction events in the session.

totals.pageviews: Total number of pageviews within the session.

totals.transactions: Total number of ecommerce transactions within the session.

totals.transactionRevenue: 	This field is deprecated. Use "totals.totalTransactionRevenue" instead (see above).


In [ ]:
"""
#standardSQL
SELECT
date,
SUM(totals.visits) AS visits,
SUM(totals.pageviews) AS pageviews,
SUM(totals.transactions) AS transactions,
SUM(totals.transactionRevenue)/1000000 AS revenue
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*`
WHERE
_TABLE_SUFFIX BETWEEN FORMAT_DATE('%Y%m%d',DATE_SUB(CURRENT_DATE(), INTERVAL 1095 DAY))
AND
FORMAT_DATE('%Y%m%d',DATE_SUB(CURRENT_DATE(), INTERVAL 1 DAY))
GROUP BY date
ORDER BY date ASC
"""

### 3 Average bounce rate per [dimension]


In [ ]:
#standardSQL

"""
SELECT
source,
total_visits,
total_no_of_bounces,
( ( total_no_of_bounces / total_visits ) * 100 ) AS bounce_rate
FROM(
SELECT
trafficSource.source AS source,
COUNT ( trafficSource.source ) AS total_visits,
SUM ( totals.bounces ) AS total_no_of_bounces,
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*`
WHERE _TABLE_SUFFIX BETWEEN '20170701' AND '20170731'
GROUP BY
source) 
ORDER BY
total_visits DESC
"""


#### Deriving Customer insights
We want to study customer and anonymous behavior across many sessions visitors have had on our website.

We are searching for behaviors that drive critical business decisions. The idea and techniques to customer segmentation are pivotal to understand customer insights.

Examples of user profiles we may be interested in:
- Users who did not make a purchase
- Users that made a purchase

### 4. Average number of product pageviews by !purchaser type! (buyer vs non-purchasers)
What was the average number of product pageviews for users who made a purchase in July 2017?

fullVisitorId: The unique visitor ID.
    
totals.pageviews: Total number of pageviews **within the session**.
    
totals.transactions: Total number of ecommerce transactions **within the session**.

In [ ]:
"""
#standardSQL
SELECT
( SUM(total_pagesviews_per_user) / COUNT(users) ) AS avg_pageviews_per_user
FROM
SELECT
fullVisitorId AS users,
SUM(totals.pageviews) AS total_pagesviews_per_user
FROM`bigquery-public-data.google_analytics_sample.ga_sessions_*`
WHERE
_TABLE_SUFFIX BETWEEN '20170701' AND '20170731'
AND
totals.transactions >=1
GROUP BY
users )
"""

**Result:**

Average number of product pageviews for users who made a purchase in July 2017
27.720954356846473

Average number of product pageviews for users who did not make a purchase in July 2017
4.1918408747077427

This informs us that people who have the intention to buy typically conduct thorough research of the site before they make a purchase.

### 5. Average number of transactions per purchase

In [ ]:
What was the average total transactions per user that made a purchase in July 2017?

In [ ]:
"""
#standardSQL
SELECT
(SUM (total_transactions_per_user) / COUNT(fullVisitorId) ) AS avg_total_transactions_per_user
FROM (
SELECT
fullVisitorId,
SUM (totals.transactions) AS total_transactions_per_user
FROM
`bigquery-public-data.google_analytics_sample.ga_sessions_*`
WHERE
_TABLE_SUFFIX BETWEEN '20170701' AND '20170731'
AND totals.transactions IS NOT NULL
GROUP BY
fullVisitorId )
"""

avg_total_transactions_per_user = 
1.1120331950207469

### 6. Average amount of money spent per session
What is the average amount of money spent per session in July 2017?

In [ ]:
"""
#standardSQL
SELECT
( SUM(total_transactionrevenue_per_user) / SUM(total_visits_per_user) ) AS
avg_revenue_by_user_per_visit
FROM (
SELECT
fullVisitorId,
SUM( totals.visits ) AS total_visits_per_user,
SUM( totals.transactionRevenue ) AS total_transactionrevenue_per_user
FROM
`bigquery-public-data.google_analytics_sample.ga_sessions_*`
WHERE
_TABLE_SUFFIX BETWEEN '20170701' AND '20170731'
AND
totals.visits > 0
AND totals.transactions >= 1
AND totals.transactionRevenue IS NOT NULL
GROUP BY
fullVisitorId )
"""

In [12]:
avg_revenue_by_user_per_visit = 120755848.69059166
print(f"Visitors that land on the Google merch site on average will generate ${avg_revenue_by_user_per_visit / pow(10, 6)}")


Customers and anons that land on the Google merch site on average will generate $120.75584869059166
